In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

from nba_api.stats.endpoints import leagueseasonmatchups
from thefuzz import fuzz, process
player_dict = get_players_pbp()
pID_dict = get_pID_pbp()

def get_missing_pId(player,player_dict):
    pId = process.extract(player,player_dict,limit=1, scorer=fuzz.partial_ratio)[0][2]
    return pId

img_DIR_P = "../data/images/players/"
fig_DIR = "../figs/players/"

In [ ]:
stats = leagueseasonmatchups.LeagueSeasonMatchups(
    league_id="00",
    per_mode_simple="Totals",
    season=get_ss(2023),
    season_type_playoffs="Playoffs"
)
df2 = stats.get_data_frames()[0]

In [ ]:
url = "https://www.basketball-reference.com/playoffs/NBA_2024_advanced.html#advanced_stats::bpm"
url = "https://www.basketball-reference.com/leagues/NBA_2024_advanced.html"
df1 = pd.read_html(url)
df1 = df1[0]
df1["playerID"] = df1["Player"].map(pID_dict)
df1.loc[df1["playerID"].isna(),"playerID"] = df1.loc[df1["playerID"].isna(),"Player"].apply(lambda x: get_missing_pId(x,player_dict))
df1["playerID"] = df1["playerID"].astype(int)
df3 = pd.merge(df2,df1[["playerID","BPM","OBPM","DBPM"]],left_on="OFF_PLAYER_ID",right_on="playerID")
df3["BPM"] = df3["BPM"].astype(float)
df3["OBPM"] = df3["OBPM"].astype(float)
df3["DBPM"] = df3["DBPM"].astype(float)
df3["pBPM"] = df3["PARTIAL_POSS"]*df3["BPM"]
df3["pOBPM"] = df3["PARTIAL_POSS"]*df3["OBPM"]
df3["pDBPM"] = df3["PARTIAL_POSS"]*df3["DBPM"]

In [ ]:
df4 = df3.groupby(["DEF_PLAYER_ID","DEF_PLAYER_NAME"])[["PARTIAL_POSS","pBPM","pOBPM","pDBPM"]].sum().reset_index()
df4["diff"] = round(df4["pBPM"]/df4["PARTIAL_POSS"],2)
df4["Odiff"] = round(df4["pOBPM"]/df4["PARTIAL_POSS"],2)
df4["Ddiff"] = round(df4["pDBPM"]/df4["PARTIAL_POSS"],2)

In [ ]:
df4.columns

In [ ]:
df5 = df4.query("PARTIAL_POSS > 50").sort_values("Odiff",ascending=False).reset_index(drop=True).reset_index()
df5["index"] +=1
df5 = df5[['index','DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'PARTIAL_POSS','Odiff']].head(10)
df5["DEF_PLAYER_ID"] = df5["DEF_PLAYER_ID"].astype(str)
df5["PARTIAL_POSS"] = df5["PARTIAL_POSS"].round(1)

In [ ]:
t = (
    GT(df5)
    .tab_header(
        title=md("Highest Matchup Difficulty: Weighted by Regular Season OBPM"),
        subtitle="Min 50 Partial Possessions Defended"
    )
    .tab_source_note(source_note="@sradjoker | source: nba.com/stats via nba_api")
    .cols_label(
        index = "#",
        DEF_PLAYER_ID = "",
        DEF_PLAYER_NAME = "Player",
        PARTIAL_POSS = "Partial Poss",
        Odiff = "Matchup Difficulty",
    )
    .fmt_image(
        columns="DEF_PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="DEF_PLAYER_NAME")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
)
t.save(fig_DIR + "Matchup_Diff_PS_2024.png",scale=3,expand=2,web_driver="edge")
t